In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier, NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from sklearn.metrics import confusion_matrix, roc_curve, auc
from pyspark.ml.stat import Correlation

# Initialize SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("UNSW_NB15_Analysis") \
    .config("spark.sql.debug.maxToStringFields", 1000) \
    .getOrCreate()

# Creating the Schema based on the given csv file
schema = StructType([
      StructField("srcip",StringType(),True),
      StructField("sport",IntegerType(),True),
      StructField("dstip",StringType(),True),
      StructField("dsport",IntegerType(),True),
      StructField("proto",StringType(),True),
      StructField("State",StringType(),True),
      StructField("dur",FloatType(),True),
      StructField("sbytes",IntegerType(),True),
      StructField("dbytes",IntegerType(),True),
      StructField("sttl",IntegerType(),True),
      StructField("dttl",IntegerType(),True),
      StructField("sloss",FloatType(),True),
      StructField("dloss",FloatType(),True),
      StructField("service",StringType(),True),
      StructField("sload",FloatType(),True),
      StructField("dload",FloatType(),True),
      StructField("spkts",IntegerType(),True),
      StructField("dpkts",IntegerType(),True),
      StructField("swin",IntegerType(),True),
      StructField("dwin",IntegerType(),True),
      StructField("stcpb",IntegerType(),True),
      StructField("dtcpb",IntegerType(),True),
      StructField("smeansz",IntegerType(),True),
      StructField("dmeansz",IntegerType(),True),
      StructField("trans_depth",IntegerType(),True),
      StructField("res_bdy_len",IntegerType(),True),
      StructField("sjit",FloatType(),True),
      StructField("djit",FloatType(),True),
      StructField("stime",StringType(),True),
      StructField("ltime",StringType(),True),
      StructField("sintpkt",FloatType(),True),
      StructField("dintpkt",FloatType(),True),
      StructField("tcprtt",FloatType(),True),
      StructField("synack",FloatType(),True),
      StructField("ackdat",FloatType(),True),
      StructField("is_sm_ips_ports",IntegerType(),True),
      StructField("ct_state_ttl",IntegerType(),True),
      StructField("ct_flw_http_mthd",IntegerType(),True),
      StructField("is_ftp_login",IntegerType(),True),
      StructField("ct_ftp_cmd",IntegerType(),True),
      StructField("ct_srv_src",IntegerType(),True),
      StructField("ct_srv_dst",IntegerType(),True),
      StructField("ct_dst_ltm",IntegerType(),True),
      StructField("ct_src_ltm",IntegerType(),True),
      StructField("ct_src_dport_ltm",IntegerType(),True),
      StructField("ct_dst_sport_ltm",IntegerType(),True),
      StructField("ct_dst_src_ltm",IntegerType(),True),
      StructField("attack_cat",StringType(),True),
      StructField("label",IntegerType(),True)])

# Load Data
df = spark.read.csv('/Users/Salma/UNSW-NB15.csv', schema=schema, header=False)

23/12/25 22:39:27 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
# Show the first few records
df.show(5)

# Summary statistics
df.describe().show()

+----------+-----+-------------+------+-----+-----+--------+------+------+----+----+-----+-----+-------+---------+---------+-----+-----+----+----+---------+----------+-------+-------+-----------+-----------+---------+---------+----------+----------+---------+--------+-------+-------+-------+---------------+------------+----------------+------------+----------+----------+----------+----------+----------+----------------+----------------+--------------+----------+-----+
|     srcip|sport|        dstip|dsport|proto|State|     dur|sbytes|dbytes|sttl|dttl|sloss|dloss|service|    sload|    dload|spkts|dpkts|swin|dwin|    stcpb|     dtcpb|smeansz|dmeansz|trans_depth|res_bdy_len|     sjit|     djit|     stime|     ltime|  sintpkt| dintpkt| tcprtt| synack| ackdat|is_sm_ips_ports|ct_state_ttl|ct_flw_http_mthd|is_ftp_login|ct_ftp_cmd|ct_srv_src|ct_srv_dst|ct_dst_ltm|ct_src_ltm|ct_src_dport_ltm|ct_dst_sport_ltm|ct_dst_src_ltm|attack_cat|label|
+----------+-----+-------------+------+-----+-----+---

+-------+-----------+------------------+-----------+------------------+-------+-------+------------------+-----------------+------------------+-----------------+-----------------+------------------+------------------+-------+--------------------+------------------+----------------+------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+-------------------+-----------------+-----------------+------------------+--------------------+--------------------+------------------+-----------------+--------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+--------------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+----------+-------------------+
|summary|      srcip|             sport|      dstip|            dsport|  proto|  State|               du

In [3]:
# Count 'None', 'NULL', '-' and NaN values in each column
for value in ['None', 'NULL', '-', None]:
    counts = df.select([count(when(col(c) == value, c)).alias(c) for c in df.columns])
    counts.show()

+-----+-----+-----+------+-----+-----+---+------+------+----+----+-----+-----+-------+-----+-----+-----+-----+----+----+-----+-----+-------+-------+-----------+-----------+----+----+-----+-----+-------+-------+------+------+------+---------------+------------+----------------+------------+----------+----------+----------+----------+----------+----------------+----------------+--------------+----------+-----+
|srcip|sport|dstip|dsport|proto|State|dur|sbytes|dbytes|sttl|dttl|sloss|dloss|service|sload|dload|spkts|dpkts|swin|dwin|stcpb|dtcpb|smeansz|dmeansz|trans_depth|res_bdy_len|sjit|djit|stime|ltime|sintpkt|dintpkt|tcprtt|synack|ackdat|is_sm_ips_ports|ct_state_ttl|ct_flw_http_mthd|is_ftp_login|ct_ftp_cmd|ct_srv_src|ct_srv_dst|ct_dst_ltm|ct_src_ltm|ct_src_dport_ltm|ct_dst_sport_ltm|ct_dst_src_ltm|attack_cat|label|
+-----+-----+-----+------+-----+-----+---+------+------+----+----+-----+-----+-------+-----+-----+-----+-----+----+----+-----+-----+-------+-------+-----------+-----------+----

+-----+-----+-----+------+-----+-----+---+------+------+----+----+-----+-----+-------+-----+-----+-----+-----+----+----+-----+-----+-------+-------+-----------+-----------+----+----+-----+-----+-------+-------+------+------+------+---------------+------------+----------------+------------+----------+----------+----------+----------+----------+----------------+----------------+--------------+----------+-----+
|srcip|sport|dstip|dsport|proto|State|dur|sbytes|dbytes|sttl|dttl|sloss|dloss|service|sload|dload|spkts|dpkts|swin|dwin|stcpb|dtcpb|smeansz|dmeansz|trans_depth|res_bdy_len|sjit|djit|stime|ltime|sintpkt|dintpkt|tcprtt|synack|ackdat|is_sm_ips_ports|ct_state_ttl|ct_flw_http_mthd|is_ftp_login|ct_ftp_cmd|ct_srv_src|ct_srv_dst|ct_dst_ltm|ct_src_ltm|ct_src_dport_ltm|ct_dst_sport_ltm|ct_dst_src_ltm|attack_cat|label|
+-----+-----+-----+------+-----+-----+---+------+------+----+----+-----+-----+-------+-----+-----+-----+-----+----+----+-----+-----+-------+-------+-----------+-----------+----

+-----+-----+-----+------+-----+-----+---+------+------+----+----+-----+-----+-------+-----+-----+-----+-----+----+----+-----+-----+-------+-------+-----------+-----------+----+----+-----+-----+-------+-------+------+------+------+---------------+------------+----------------+------------+----------+----------+----------+----------+----------+----------------+----------------+--------------+----------+-----+
|srcip|sport|dstip|dsport|proto|State|dur|sbytes|dbytes|sttl|dttl|sloss|dloss|service|sload|dload|spkts|dpkts|swin|dwin|stcpb|dtcpb|smeansz|dmeansz|trans_depth|res_bdy_len|sjit|djit|stime|ltime|sintpkt|dintpkt|tcprtt|synack|ackdat|is_sm_ips_ports|ct_state_ttl|ct_flw_http_mthd|is_ftp_login|ct_ftp_cmd|ct_srv_src|ct_srv_dst|ct_dst_ltm|ct_src_ltm|ct_src_dport_ltm|ct_dst_sport_ltm|ct_dst_src_ltm|attack_cat|label|
+-----+-----+-----+------+-----+-----+---+------+------+----+----+-----+-----+-------+-----+-----+-----+-----+----+----+-----+-----+-------+-------+-----------+-----------+----

+-----+-----+-----+------+-----+-----+---+------+------+----+----+-----+-----+-------+-----+-----+-----+-----+----+----+-----+-----+-------+-------+-----------+-----------+----+----+-----+-----+-------+-------+------+------+------+---------------+------------+----------------+------------+----------+----------+----------+----------+----------+----------------+----------------+--------------+----------+-----+
|srcip|sport|dstip|dsport|proto|State|dur|sbytes|dbytes|sttl|dttl|sloss|dloss|service|sload|dload|spkts|dpkts|swin|dwin|stcpb|dtcpb|smeansz|dmeansz|trans_depth|res_bdy_len|sjit|djit|stime|ltime|sintpkt|dintpkt|tcprtt|synack|ackdat|is_sm_ips_ports|ct_state_ttl|ct_flw_http_mthd|is_ftp_login|ct_ftp_cmd|ct_srv_src|ct_srv_dst|ct_dst_ltm|ct_src_ltm|ct_src_dport_ltm|ct_dst_sport_ltm|ct_dst_src_ltm|attack_cat|label|
+-----+-----+-----+------+-----+-----+---+------+------+----+----+-----+-----+-------+-----+-----+-----+-----+----+----+-----+-----+-------+-------+-----------+-----------+----

In [4]:
# Data cleaning
df = df.na.fill(0)
df = df.na.replace(['None', 'NULL', '-'], [None, None, None])
df = df.withColumn('attack_cat', trim(col("attack_cat")))

In [5]:
# Fill missing values in 'attack_cat' with 'Normal'
df = df.fillna({'attack_cat':'Normal'})

In [6]:
# Exploratory Data Analysis
df.select("attack_cat").describe().show()

distinct_attack_categories = df.select('attack_cat').distinct().collect()
print("Distinct Attack Categories:", distinct_attack_categories)

+-------+----------+
|summary|attack_cat|
+-------+----------+
|  count|   2539739|
|   mean|      NULL|
| stddev|      NULL|
|    min|  Analysis|
|    max|     Worms|
+-------+----------+



Distinct Attack Categories: [Row(attack_cat='Worms'), Row(attack_cat='Shellcode'), Row(attack_cat='Fuzzers'), Row(attack_cat='Analysis'), Row(attack_cat='DoS'), Row(attack_cat='Reconnaissance'), Row(attack_cat='Backdoor'), Row(attack_cat='Exploits'), Row(attack_cat='Normal'), Row(attack_cat='Generic'), Row(attack_cat='Backdoors')]


In [7]:
# Fill missing values in 'attack_cat' with 'Normal'
df = df.fillna({'attack_cat':'Normal'})

In [8]:
# Count of each attack category
df.groupBy('attack_cat').count().orderBy('count', ascending=False).show()

+--------------+-------+
|    attack_cat|  count|
+--------------+-------+
|        Normal|2218456|
|       Generic| 215481|
|      Exploits|  44525|
|       Fuzzers|  24246|
|           DoS|  16353|
|Reconnaissance|  13987|
|      Analysis|   2677|
|      Backdoor|   1795|
|     Shellcode|   1511|
|     Backdoors|    534|
|         Worms|    174|
+--------------+-------+



In [ ]:
# Plotting function improved for reusability
def plot_pie_chart(data, title, column):
    counts = data.groupBy(column).count().collect()
    labels, values = zip(*[(row[column], row['count']) for row in counts])
    plt.figure(figsize=(10, 6))
    plt.pie(values, labels=labels, autopct='%1.2f%%')
    plt.title(title)
    plt.show()

plot_pie_chart(df, "Attack Type Distribution", "attack_cat")


In [ ]:
def plot_pie_chart(data, title):
    labels, counts = zip(*[(row['Label'], row['count']) for row in data.collect()])
    plt.figure(figsize=(7, 7))
    plt.pie(counts, labels=labels, autopct='%1.2f%%')
    plt.title(title)
    plt.show()
    
plot_pie_chart(df.groupBy("Label").count(), 'Label Count Distribution')

In [ ]:
from scipy.stats import ttest_ind
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Assuming df is your PySpark DataFrame
pandas_df = df.select("sbytes", "dbytes").toPandas()

# T-test
t_stat, p_value = ttest_ind(pandas_df["sbytes"], pandas_df["dbytes"])
print("T-test p-value:", p_value)

# Kernel Density Estimation
sns.kdeplot(pandas_df["sbytes"])
plt.title("Kernel Density Estimation for sbytes")
plt.xlabel("sbytes values")
plt.ylabel("Density")
plt.show()

In [ ]:
# Correlation Analysis

In [ ]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler
import seaborn as sns
import matplotlib.pyplot as plt

# List of column names for correlation analysis
correlation_columns = ["sport", "dsport", "dur", "sbytes", "dbytes", "sttl", "dttl", "sloss", "dloss",
        "sload", "dload", "spkts", "dpkts", "swin", "dwin", "stcpb", "dtcpb", "smeansz", 
        "dmeansz", "trans_depth", "res_bdy_len", "sjit", "djit", "sintpkt", "dintpkt",
        "tcprtt", "synack", "ackdat", "is_sm_ips_ports", "ct_state_ttl", "ct_flw_http_mthd",
        "is_ftp_login", "ct_ftp_cmd", "ct_srv_src", "ct_srv_dst", "ct_dst_ltm", "ct_src_ltm",
        "ct_src_dport_ltm", "ct_dst_sport_ltm", "ct_dst_src_ltm", "label"]

# Assembling features for correlation analysis
assembler = VectorAssembler(inputCols=correlation_columns, outputCol="corr_features")
df_vector = assembler.transform(df).select("corr_features")

In [ ]:
# Calculating the correlation matrix
correlation_matrix = Correlation.corr(df_vector, "corr_features").collect()[0][0].toArray()

In [ ]:
# Plotting heatmap of correlation matrix
plt.figure(figsize=(30, 30))
sns.heatmap(correlation_matrix, annot=True)
plt.show()

In [ ]:
df_corr_matrix = spark.createDataFrame(correlation_matrix, correlation_columns)
# Show correlation between 'sport' and 'Label'
df_corr_matrix.select('sport', 'Label').show()

In [ ]:
from pyspark.sql.functions import col
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.classification import RandomForestClassifier, DecisionTreeClassifier, NaiveBayes
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.ml import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Preparing Data for Machine Learning
input_columns = ["sport", "dsport", "dur", "sbytes", "dbytes", "sttl", "dttl", "sloss", "dloss",
        "sload", "dload", "spkts", "dpkts", "swin", "dwin", "stcpb", "dtcpb", "smeansz", 
        "dmeansz", "trans_depth", "res_bdy_len", "sjit", "djit", "sintpkt", "dintpkt",
        "tcprtt", "synack", "ackdat", "is_sm_ips_ports", "ct_state_ttl", "ct_flw_http_mthd",
        "is_ftp_login", "ct_ftp_cmd", "ct_srv_src", "ct_srv_dst", "ct_dst_ltm", "ct_src_ltm",
        "ct_src_dport_ltm", "ct_dst_sport_ltm", "ct_dst_src_ltm", "label"]

assembler = VectorAssembler(inputCols=input_columns, outputCol="features")
transformed_df = assembler.transform(df)

In [ ]:
# Create a DataFrame for the model
model_data_df = transformed_df.select("features", "Label")

In [ ]:
# Splitting the data into training and test sets
train_df, test_df = model_data_df.randomSplit([0.7, 0.3])

In [ ]:
# Random Forest Classifier

In [ ]:
# Train Random Forest Classifier
rf_classifier = RandomForestClassifier(labelCol="Label", numTrees=50)
rf_model = rf_classifier.fit(train_df)
rf_predictions = rf_model.transform(test_df)

In [ ]:
# Evaluate accuracy and AUC
rf_evaluator = BinaryClassificationEvaluator(labelCol="Label")
rf_accuracy = rf_evaluator.evaluate(rf_predictions)
rf_auc = rf_evaluator.setMetricName("areaUnderROC").evaluate(rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy}, AUC: {rf_auc}")

In [ ]:
# Feature importances
print("Random Forest Feature Importances:", rf_model.featureImportances)

In [ ]:
# Confusion Matrix for Random Forest
y_true_rf = [row['label'] for row in rf_predictions.select("label").collect()]
y_pred_rf = [row['prediction'] for row in rf_predictions.select("prediction").collect()]
conf_matrix_rf = confusion_matrix(y_true_rf, y_pred_rf)
plt.figure(figsize=(8, 8))
sns.heatmap(conf_matrix_rf, annot=True, fmt='g')
plt.title('Random Forest Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()

In [ ]:
from pyspark.ml.linalg import DenseVector

y_true = [row['Label'] for row in rf_predictions.select("Label").collect()]
y_prob = [row['probability'] for row in rf_predictions.select("probability").collect()]
y_prob_positive = [prob[1] for prob in y_prob if isinstance(prob, DenseVector)]
fpr, tpr, thresholds = roc_curve(y_true, y_prob_positive)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Decision Tree Classifier for Multi-Class Classification
indexer = StringIndexer(inputCol="attack_cat", outputCol="attack_cat_index")
df_indexed = indexer.fit(df).transform(df)
assembler = VectorAssembler(inputCols=input_columns, outputCol="features")
multi_model_data = assembler.transform(df_indexed).select("features", "attack_cat_index")

In [ ]:
# Splitting the data
train_df, test_df = multi_model_data.randomSplit([0.7, 0.3])

In [ ]:
# Train multi-class Decision Tree Classifier
dt_multi_classifier = DecisionTreeClassifier(labelCol="attack_cat_index").fit(train_df)
dt_predictions = dt_multi_classifier.transform(train_df)

In [ ]:
# Evaluate multi-class classifier
multi_evaluator = MulticlassClassificationEvaluator(labelCol="attack_cat_index")
dt_accuracy = multi_evaluator.setMetricName("accuracy").evaluate(dt_predictions)
dt_precision = multi_evaluator.setMetricName("weightedPrecision").evaluate(dt_predictions)
dt_recall = multi_evaluator.setMetricName("weightedRecall").evaluate(dt_predictions)

print(f"Decision Tree Accuracy: {dt_accuracy}, Precision: {dt_precision}, Recall: {dt_recall}")

In [ ]:
# Naive Bayes Classifier
nb_classifier = NaiveBayes(labelCol="attack_cat_index", featuresCol="features")
nb_pipeline = Pipeline(stages=[indexer, assembler, nb_classifier])
nb_model = nb_pipeline.fit(train_df)
nb_predictions = nb_model.transform(test_df)

In [ ]:
# Evaluate Naive Bayes Model
nb_evaluator = MulticlassClassificationEvaluator(labelCol="attack_cat_index", predictionCol="prediction")
nb_accuracy = nb_evaluator.setMetricName("accuracy").evaluate(nb_predictions)
nb_precision = nb_evaluator.setMetricName("weightedPrecision").evaluate(nb_predictions)
nb_recall = nb_evaluator.setMetricName("weightedRecall").evaluate(nb_predictions)
print(f"Naive Bayes Accuracy: {nb_accuracy}, Precision: {nb_precision}, Recall: {nb_recall}")

In [ ]:
# Confusion Matrix for Naive Bayes
y_true_nb = [row['attack_cat_index'] for row in nb_predictions.select("attack_cat_index").collect()]
y_pred_nb = [row['prediction'] for row in nb_predictions.select("prediction").collect()]
conf_matrix_nb = confusion_matrix(y_true_nb, y_pred_nb)
plt.figure(figsize=(10, 10))
sns.heatmap(conf_matrix_nb, annot=True, fmt='g')
plt.title('Naive Bayes Confusion Matrix')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

In [ ]:
# Clean up
spark.stop()